In [2]:
import numpy as np
import time
import pygame

def create_rot_matrix(angle: float):
    return np.array([[np.cos(angle), np.sin(angle)],
                    [-np.sin(angle), np.cos(angle)]])

class Robot:
    length = None
    width = None
    corner_angle = None
    half_diag_length = None

    current_pos = None
    current_speed = 0
    current_angular_velocity = 0
    sensor_vals = [0, 0]
    centre_of_rot = None

    angle = None
    direction_unit_vec = None

    corners = None
    corner_offsets = None

    dist_travelled = 0

    def __init__(self, dimensions: tuple, start_pos: tuple, angle: float):
        self.width = dimensions[1]
        self.length = dimensions[0]

        self.corner_angle = np.arctan(self.width/self.length) #TThis is the angle b/w direction vector and centre->corner vector
        self.half_diag_length = np.linalg.norm([self.width/2, self.length/2]) #Half diagonal length is useful in calculations

        self.current_pos = np.array(start_pos, dtype='float64')
        self.angle = angle

        self.direction_unit_vec = create_rot_matrix(angle) @ np.array([1, 0])

        corner_0_offset = create_rot_matrix(angle + self.corner_angle) @ (self.half_diag_length * np.array([1, 0]))
        corner_1_offset = create_rot_matrix(angle - self.corner_angle) @ (self.half_diag_length * np.array([1, 0]))
        self.corner_offsets = np.array([corner_0_offset, corner_1_offset])

        self.corners = np.zeros((4, 2))
        self.corners[0] = self.current_pos + self.corner_offsets[0]
        self.corners[1] = self.current_pos + self.corner_offsets[1]
        self.corners[2] = self.current_pos - self.corner_offsets[0]
        self.corners[3] = self.current_pos - self.corner_offsets[1]

        self.wheel_pos = np.zeros((2, 2))
        self.wheel_pos[0] = (self.corners[0] + self.corners[3])/2
        self.wheel_pos[1] = (self.corners[1] + self.corners[2])/2

        ################################################
        #   Direction unit vector (init. dir = pi/2)   #
        #        D                                     #
        #        |                                     #
        #        |                                     #
        #   X,---|---,X Corner Offsets (O -> X)        #
        #    |\  |  /|                                 #
        #    | \ | / |                                 #
        #    |  \|/  |                                 #
        #    |   O   |  Length                         #
        #    |  pos  |                                 #
        #    |       |                 ^ Y             #
        #    |       |                 |               #
        #    `-------'                 |               #
        #      Width                   '-----> X       #
        ################################################


    # Helper functions
    # DO NOT TOUCH THESE
    def rotate(self, rot_angle: float):
        # TODO!!!!! USE CENTRE_OF_ROT TO CHANGE

        r_matrix = create_rot_matrix(rot_angle)

        for idx in range(4):
            self.corners[idx] = r_matrix @ (self.corners[idx] - self.centre_of_rot) + self.centre_of_rot

        for idx in range(2):
            self.wheel_pos[idx] = r_matrix @ (self.wheel_pos[idx] - self.centre_of_rot) + self.centre_of_rot

        self.current_pos = r_matrix @ (self.current_pos - self.centre_of_rot) + self.centre_of_rot

        self.corner_offsets[0] = self.corners[0] - self.current_pos
        self.corner_offsets[1] = self.corners[1] - self.current_pos

        tmp = (self.corners[0] + self.corners[1])/2 - self.current_pos
        self.direction_unit_vec = tmp/np.linalg.norm(tmp)

    def move(self, displacement: float):
        self.current_pos += displacement
        for idx in range(4):
            self.corners[idx] += displacement

        self.wheel_pos[0] += displacement
        self.wheel_pos[1] += displacement

    # Emulator functions
    # You only ever have to use these functions to update the robots condition inside the emulator
    # The function parameters are taken from the robot interface
    # The robot interface preprocesses the data to provide speed values
    # DO NOT LET PARTICIPANTS DIRECTLY ACCESS THESE

    def update_pos(self, time_elapsed: float, radius_of_rot_div_w):
        if(not radius_of_rot_div_w == 'INF'):
            self.centre_of_rot = self.current_pos + radius_of_rot_div_w * 2 * (self.wheel_pos[0] - self.current_pos) # WARNING IS IT WITH RESPECT TO 1 OR 0
            self.rotate(self.current_angular_velocity * time_elapsed)
            self.dist_travelled += (self.current_angular_velocity * time_elapsed) * radius_of_rot_div_w * self.width
        else:
            self.centre_of_rot = 'INF'
            displacement = self.direction_unit_vec * self.current_speed * time_elapsed
            self.move(displacement)
            self.dist_travelled += np.linalg.norm(displacement)

    def set_speed(self, speed: float):
        self.current_speed = speed

    def set_ang_vel(self, ang_vel: float):
        self.current_angular_velocity = ang_vel

    def get_sensor_vals(self, screen: pygame.surface.Surface):
        '''
        reads the IR sensor and return a list
        SHOULD BE USED BEFORE UPDATING THE ROBOT ON SCREEN


        sensor_vals[0] -> left sensor
        sensor_vals[1] -> right sensor
        (assuming corners[0] is front left corner)

        '''
        colour1 = screen.get_at((int(self.corners[0][0]), int(self.corners[0][1])))
        colour1_gs = (colour1[0] + colour1[1] + colour1[2]) / 3
        colour2 = screen.get_at((int(self.corners[1][0]), int(self.corners[1][1])))
        colour2_gs = (colour2[0] + colour2[1] + colour2[2]) / 3
        if (colour1_gs < 150):
            self.sensor_vals[0] = 0
        else:
            self.sensor_vals[0] = 1
        if (colour2_gs < 150):
            self.sensor_vals[1] = 0
        else:
            self.sensor_vals[1] = 1

        return self.sensor_vals
    

In [26]:
import numpy as np

# This class will be used by the participants to exchange data with the emulator
# The signal setter functions need to be changed to use the motor speed values
# This class helps

MOTOR_MAX_RPM = 60 # in rpm, which is 120pi rad/min, which is 2pi rad/s, which corresponds to 15.7 cm/s
WHEEL_RADIUS = 25 #in px, 2.5 in cm
ACCELERATION = 1 # pixel per tick
ANGULAR_ACCELERATION = 0.3 # per tick

class Motor:
    duty_cycle = 0
    rpm = 0
    wheel_speed = 0
    wheel_angular_speed = 0
    in1 = False
    in2 = False

    def __init__(self, position: int, d_cycle: int, IN1_PIN_val: bool, IN2_PIN_val: bool):
        self._rpm : int = 0
        self.position = position # LEFT = 0, RIGHT = 1
        self.write_motor_pins(d_cycle, IN1_PIN_val, IN2_PIN_val)

#     @property
#     def rpm(self):
#         return self._rpm
#
#     @rpm.setter
#     def rpm(self, value : int):
#         if (type(value) == int):
#             self._rpm = value
#             if (value > MOTOR_MAX_RPM):
#                 self._rpm = MOTOR_MAX_RPM
#         else:
#             raise(f"RPM can only be a integer. It was tried to be set to {value} which is of type {type(value)}")

    def set_direction_pin_vals(self, IN1_PIN_val: bool, IN2_PIN_val: bool):
        self.in1 = IN1_PIN_val
        self.in2 = IN2_PIN_val

        if(self.in1 == self.in2):
            self.direction = 0
        elif(self.in1):
            if(self.position == 0):
                self.direction = -1
            else:
                self.direction = 1
        elif(self.in2):
            if(self.position == 0):
                self.direction = 1
            else:
                self.direction = -1

    def set_duty_cycle(self, d_cycle: int):
        # 0 -> 255 :: 0 -> MOTOR_MAX_RPM :: 0T -> 100%T
        self.duty_cycle = d_cycle
        self.rpm = (self.direction * d_cycle * MOTOR_MAX_RPM)/255
        self.wheel_angular_speed = self.rpm * 2 * np.pi / 60 # in rad/s
        self.wheel_speed = self.wheel_angular_speed * WHEEL_RADIUS # in px/s

    def write_motor_pins(self, d_cycle: int, IN1_PIN_val: bool, IN2_PIN_val: bool):
        self.set_direction_pin_vals(IN1_PIN_val, IN2_PIN_val)
        self.set_duty_cycle(d_cycle)

class RobotInterface:
    '''
    - motor1speed: float IN
    - motor2speed: float IN
    - speed: float SIGNAL
    - ang_vel: float SIGNAL
    - sensor_values: [bool, bool] OUT
    '''
    is_decel = False
    is_accel = False
    new_speed = None
    current_speed = 0

    motors = None
    width = None
    length = None
    radius_of_rotation_div_w = None

    is_ang_decel = False
    is_ang_accel = False
    new_ang_vel = None
    current_angular_velocity = 0

    prev_signals = None

    def __init__(self, signals, dimensions: tuple):
        m_left = Motor(0, signals[0][0], signals[0][1], signals[0][2])
        m_right = Motor(1, signals[1][0], signals[1][1], signals[1][2])
        self.motors = [m_left, m_right]
        self.width = dimensions[1]
        self.length = dimensions[0]
        self.prev_signals = signals

    ############################################
    # DONT TOUCH THESE WHILE NOT DEBUGGING
    def set_speed(self, speed):
        ############ !!!!!!!!!!! #####################
        if(self.current_speed <= speed):
            self.is_accel = True
            self.is_decel = False
            self.new_speed = speed
        else:
            self.is_accel = False
            self.is_decel = True
            self.new_speed = speed
        ############ !!!!!!!!!!! #####################

        # self.current_speed = speed

    def set_ang_vel(self, ang_vel):
        ########### !!!!!!!!!!! #####################
        if(self.current_angular_velocity <= ang_vel):
            self.is_ang_accel = True
            self.is_ang_decel = False
            self.new_ang_vel = ang_vel
        else:
            self.is_ang_accel = False
            self.is_ang_decel = True
            self.new_ang_vel = ang_vel
        ########### !!!!!!!!!!! #####################

        # self.current_angular_velocity = ang_vel

    def accel_decel(self, ticks_elapsed):
        pass
        if(self.is_accel):
            if(self.current_speed >= self.new_speed):
                self.current_speed = self.new_speed
                self.new_speed = None
                self.is_accel = False
                self.is_decel = False
                print("FIN_ACCEL")
                # print("NEW SPEEEED" + self.new_speed)

            else:
                self.current_speed += ACCELERATION * ticks_elapsed
                print("IS_ACCEL")
                # print("NEW SPEEEED",self.new_speed)
        if(self.is_decel):
            if(self.current_speed <= self.new_speed):
                self.current_speed = self.new_speed
                self.new_speed = None
                self.is_decel = False
                self.is_accel = False
                print("FIN_DECEL")
                # print("NEW SPEEEED" + self.new_speed)

            else:
                self.current_speed -= ACCELERATION * ticks_elapsed
                print("IS_DECEL")

        if(self.is_ang_accel):
            if(self.current_angular_velocity >= self.new_ang_vel):
                self.current_angular_velocity = self.new_ang_vel
                self.new_ang_vel = None
                self.is_ang_accel = False
                print("FIN_ANG_ACCEL")
                # print("NEW SPEEEED" + self.new_speed)

            else:
                self.current_angular_velocity += ANGULAR_ACCELERATION * ticks_elapsed
                print("IS_ANG_ACCEL")

        if(self.is_ang_decel):
            if(self.current_angular_velocity <= self.new_ang_vel):
                self.current_angular_velocity = self.new_ang_vel
                self.new_ang_vel = None
                self.is_ang_decel = False
                print("FIN_ANG_DECEL")
                # print("NEW SPEEEED" + self.new_speed)

            else:
                self.current_angular_velocity -= ANGULAR_ACCELERATION * ticks_elapsed
                print("IS_ANG_ACCEL")
    ############################################

    def update_signals(self, signals: tuple):
        if(signals != self.prev_signals):
            self.motors[0].write_motor_pins(signals[0][0], signals[0][1], signals[0][2])
            self.motors[1].write_motor_pins(signals[1][0], signals[1][1], signals[1][2])
            self.set_speed((self.motors[0].wheel_speed + self.motors[1].wheel_speed)/2)
            self.set_ang_vel((self.motors[1].wheel_speed - self.motors[0].wheel_speed)/self.width)

            if(not (self.motors[1].wheel_angular_speed == self.motors[0].wheel_angular_speed)):
                self.radius_of_rotation_div_w = ((self.motors[1].wheel_angular_speed + self.motors[0].wheel_angular_speed)/(2*(self.motors[1].wheel_angular_speed - self.motors[0].wheel_angular_speed)))
                print(self.radius_of_rotation_div_w)
            else:
                self.radius_of_rotation_div_w = 'INF'
            self.prev_signals = signals

    def get_speed(self):
        return self.current_speed

    def get_ang_vel(self):
        return self.current_angular_velocity


# Pygame implementation of environment

In [4]:
# Create map as string
# Can get map from files as string
# Made using asciiflow.comn

map = '''


                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
                   │          │                       │                                      │
   ────────────────┤          │                       │        ───┬──────────┬───────────────┤
                   │          │                       │           │          │               │
                   │          │                       │           │          │               │
                   │          │                       │           │                          │
                   │          │                       │           │                          ├───────────┐
                   │          ├──────────────         │           │                          │           │
                   │          │                       │           ├──────────────────────    │           │
                   │          │                       │           │                          │           │
   ────────────────┤          │                       │           │                          │           │
                   │          │                       │                                      │           │
                   │          │                       │                                      │           │
                   │          │                       │                                      │           │
                   │          │                       │                                      │           │
                   ├──────────┴─────────────┬─────────┴─────────┬────────────────────────────┴───────────┴───┬────
                   │                        │                   │                                            │
                   │                        │                   │                                            │
                   │                        │                   │                      │                     │
                   │         │              │                   │                      │                     │
                   │         │              │                   ├────────────┬─────────┴──────               │
                   ├─────────┴────          │                   │            │                               │
                   │                        │                   │            │                               │
                   │                        ├────────────┐      │            │                               │
   ────────────────┤               │        │            │      │            │                               │
                   │               │        │            │      │            │
                   │               │        │            │      │            │
                   │               │        S            │      │            ├──────────────────────────────────
                   │               │                     │      │            │
                   │               │                     │      │            │
                   │               │                     │      │            │
                   │               │                     │      │            ├─────────┬──────────────┬───────────
                   │               │                     │      │            │         │              │
                   │               │                     │      │            │         │              │
                   ├───────────────┴─────────────────────┘      │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         │              │
                   │                                            │            │         └──────────────┴──────────G
                   │                                            │            │
                   │                                            │            │
                   │                                            │            │
                   │                                            │            │
                   │                                            │            │

'''

# Convert string to rows of strings for easier iteration and position access
map_array = map.split('\n')

In [31]:
import numpy as np
import pygame
import csv
import copy
import os
# from robot import *
# from robotinterface import *
# from mazemap import *

SCREEN_WIDTH = 1600#1400
SCREEN_HEIGHT = 1200 #780
strip_width = 15 #1.5cm
shared_file = "shared.txt"

ROBOT_WIDTH = 80 #8cm
ROBOT_LENGTH = 80 #8cm
##########################
# SCALE: 1cm = 10px
##########################

# Determine the start pos
start_pos = (0, 0)
for row in map_array:
    if 'S' in row:
        start_pos = np.array([row.index('S'), map_array.index(row)])
        print(start_pos)

# Temp variable Used once to center the robot onto the path at the start
path_offset = np.array([strip_width/2, strip_width/2])
# Create the robot object (Dimensions, start position, Direction facing)
my_rob = Robot((ROBOT_LENGTH, ROBOT_WIDTH), strip_width*start_pos + path_offset, 0)
# signal_list = [[255, False, True], [255, True, False]]
robot_interface = RobotInterface(signal_list, (ROBOT_LENGTH, ROBOT_WIDTH))

# Initialize the pygame objects and screen
pygame.init()
pygame.font.init()
my_font = pygame.font.SysFont('Roboto', 30)

clock = pygame.time.Clock()

screen = pygame.display.set_mode([SCREEN_WIDTH, SCREEN_HEIGHT])
running = True

file_poll_time_period = 50
time_since_last_poll = 0
latest_modified_time = 0

while running:
    screen.fill((200, 200, 200)) #Fill background

    elapsed_time = clock.get_time()
    time_since_last_poll += elapsed_time
    if(time_since_last_poll > file_poll_time_period and os.stat('shared.txt').st_mtime > latest_modified_time):
        latest_modified_time = os.stat('shared.txt').st_mtime
        time_since_last_poll = 0
        with open('shared.txt', newline='') as csvfile:
            reader_obj = csv.reader(csvfile, delimiter=',', quotechar='|')
            wheel_no = 0
            for row in reader_obj:
                for i in range(len(row)):
                    row[i] = int(row[i])
                if(wheel_no < 2):
                    signal_list[wheel_no] = copy.deepcopy(row)
                    wheel_no += 1
        print(pygame.time.get_ticks())
        

    #######################################################################################################################
    #######################################################################################################################
    # 1.                                                                                                                 ##
    # Drawing the path on screen                                                                                         ##
    for event in pygame.event.get():                                                                                     ##
        if event.type == pygame.QUIT:                                                                                    ##
            running = False                                                                                              ##
                                                                                                                         ##
    # Choosing j, i (position of block) and filling it in if it's a strip                                                ##
    for j in range(len(map_array)):                                                                                      ##
        row = map_array[j]                                                                                               ##
        for i in range(len(row)):                                                                                        ##
            block_pos = (i*strip_width, j*strip_width) #Get block position                                               ##
            if(row[i] in '─,│,┐,┘,└,┌,┬,┤,┴,├,┼'):                                                                       ##
                pygame.draw.rect(screen, (0, 0, 0), block_pos + (strip_width, strip_width)) # Draw the path at block_pos ##
            elif row[i] == 'S':                                                                                          ##
                pygame.draw.rect(screen, (200, 200, 0), block_pos + (strip_width, strip_width)) # Draw start state       ##
            elif row[i] == 'G':                                                                                          ##
                pygame.draw.rect(screen, (0, 200, 0), block_pos + (strip_width, strip_width)) # Draw goal state          ##
                                                                                                                         ##
    my_rob.get_sensor_vals(screen)                                                                                       ##
                                                                                                                         ##
    text_surface = my_font.render("Time elapsed: " + str(pygame.time.get_ticks()), False, (0, 0, 0))                     ##
    screen.blit(text_surface, (300,0))                                                                                   ##
    txt_s = my_font.render("AngSpeed: " + str((my_rob.current_angular_velocity)), False, (0, 0, 0))                                 ##
    screen.blit(txt_s, (600,0))                                                                                          ##
    txt_s = my_font.render("Dist: " + str(int(my_rob.dist_travelled)), False, (0, 0, 0))                                 ##
    screen.blit(txt_s, (800,0))                                                                                          ##
    txt_s = my_font.render("Sensor vals: " + str(my_rob.sensor_vals), False, (0, 0, 0))                                  ##
    screen.blit(txt_s, (1100,0))                                                                                         ##
                                                                                                                         ##
    #######################################################################################################################
    #######################################################################################################################
    # 2.                                                                                                                 ##
    # Updating signals and sensor values (DO NOT CHANGE THE DRAW ORDER)                                                  ##
                                                                                                                         ##
    robot_interface.update_signals(tuple(signal_list))                                                                   ##
    robot_interface.accel_decel(elapsed_time)                                                                            ##
                                                                                                                         ##
    # !!!!!!!!!!!!!!!!! ####################################################################                             ##
    # DO NOT TOUCH THIS ####################################################################                             ##
    my_rob.set_speed(robot_interface.get_speed())                                        ###                             ##
    my_rob.set_ang_vel(robot_interface.get_ang_vel())                                    ###                             ##
    my_rob.update_pos(elapsed_time/1000, robot_interface.radius_of_rotation_div_w)       ###                             ##
    # DO NOT TOUCH THIS ####################################################################                             ##
    # !!!!!!!!!!!!!!!!! ####################################################################                             ##
                                                                                                                         ##
    # DEBUG ########################################################                                                     ##
    # my_rob.get_sensor_vals(screen)                                 #                                                     ##
    # if(my_rob.sensor_vals[0] == 0 and my_rob.sensor_vals[1] == 0): #                                                     ##
    #     signal_list = [[255, False, True], [255, False, True]]     #                                                     ##
    # DEBUG ########################################################                                                     ##
                                                                                                                         ##
    #######################################################################################################################
    #######################################################################################################################
    # 3.                                                                                                                 ##
    # Drawing the robot pulygon and wheels                                                                               ##
    pygame.draw.polygon(screen, (0, 100, 10), my_rob.corners, width=0)                                                   ##
    for c_idx in range(4):                                                                                               ##
        if c_idx < 2:                                                                                                    ##
            pygame.draw.circle(screen, (255, 255, 0), my_rob.corners[c_idx], 5)                                          ##
        else:                                                                                                            ##
            pygame.draw.circle(screen, (0, 0, 0), my_rob.corners[c_idx], 5)                                              ##
                                                                                                                         ##
    pygame.draw.circle(screen, (0, 0, 0), my_rob.wheel_pos[0], 5)                                                        ##
    pygame.draw.circle(screen, (0, 0, 0, 0), my_rob.wheel_pos[1], 5)                                                     ##
    pygame.draw.circle(screen, (255, 0, 0), my_rob.current_pos, 3)                                                       ##
    pygame.draw.circle(screen, (0, 0, 0), my_rob.current_pos + 30*my_rob.direction_unit_vec, 3)                          ##
    if(type(my_rob.centre_of_rot) == np.ndarray):                                                                        ##
        pygame.draw.circle(screen, (255, 0, 255), my_rob.centre_of_rot, 3)                                               ##
    pygame.display.flip()                                                                                                ##
                                                                                                                         ##
    pygame.display.set_caption(f'Current FPS: {str(clock.get_fps())}')                                                   ##
    clock.tick(60)                                                                                                       ##
                                                                                                                         ##
    #######################################################################################################################
    #######################################################################################################################
pygame.quit()


[44 35]
-0.5
IS_ACCEL
IS_ANG_ACCEL
628
IS_ACCEL
FIN_ANG_ACCEL
IS_ACCEL
IS_ACCEL
IS_ACCEL
IS_ACCEL
FIN_ACCEL
1357
-0.5
IS_DECEL
IS_ANG_ACCEL
IS_DECEL
FIN_ANG_DECEL
IS_DECEL
IS_DECEL
IS_DECEL
FIN_DECEL
2167
IS_ACCEL
IS_ANG_ACCEL
IS_ACCEL
FIN_ANG_ACCEL
IS_ACCEL
IS_ACCEL
IS_ACCEL
FIN_ACCEL
2958
-0.5
IS_DECEL
IS_ANG_ACCEL
IS_DECEL
FIN_ANG_DECEL
IS_DECEL
IS_DECEL
IS_DECEL
FIN_DECEL
3764
IS_ACCEL
IS_ANG_ACCEL
IS_ACCEL
FIN_ANG_ACCEL
IS_ACCEL
IS_ACCEL
IS_ACCEL
FIN_ACCEL
4572
-0.5
IS_DECEL
IS_ANG_ACCEL
IS_DECEL
FIN_ANG_DECEL
IS_DECEL
IS_DECEL
IS_DECEL
FIN_DECEL


In [6]:
import os
filename = "black.jpg"
os.stat(filename).st_mtime

1715084937.6889305